In [9]:
class Pipe:
    def __init__(self, value):
        self.value = value
    
    def __rshift__(self, func):
        return Pipe(func(self.value))
    
    def __or__(self, func):
        return Pipe(func(self.value))
    
# pipe = Pipe([1,2,3]) >> sum >> (lambda x: x/2)
pipe = Pipe([1,2,3]) | sum | (lambda x: x/2)
pipe.value

3.0

In [10]:
class Pipe:
    def __init__(self, funcs:list):
        self.funcs = funcs or []

    def run(self, value):
        result = value
        for func in self.funcs:
            result = func(result)
        return result

In [13]:
import numpy as np
pipe = Pipe(funcs=[np.sqrt, (lambda x: x**5), np.log])
pipe.run([1,2,3])

array([0.        , 1.73286795, 2.74653072])

In [ ]:
import time
class Timer:
    def __enter__(self):
        self.start = time.time()
    
    def __exit__(self, *args):
        end = time.time()
        print("{elapsed}".format(elapsed=end-self.start))

with Timer():
    time.sleep(5)

5.000343322753906


In [25]:
import os
os.environ['DEBUG'] = 'true'
os.environ['API_KEY'] = 'test'
class TempEnv:
    def __init__(self, **kwargs):
        self.env_vars = kwargs
        self.old_vars = {}

    def __enter__(self):
        for key, value in self.env_vars.items():
            self.old_vars[key]=os.getenv(key)
            os.environ[key] = value

    def __exit__(self, *args):
        for key, value in self.old_vars.items():
            if value:
                os.environ[key]=value
            else:
                os.environ.pop(key, None)

print(os.environ['DEBUG'])
print(os.environ['API_KEY'])
with TempEnv(DEBUG='false', API_KEY='tost'):
    print(os.environ['DEBUG'])
    print(os.environ['API_KEY'])

print(os.environ['DEBUG'])
print(os.environ['API_KEY'])

true
test
false
tost
true
test


In [ ]:
from typing import Any, Dict
class Config:
    _instance = None
    
    def __init__(self):
        self.settings: Dict[str, Any] = {"debug": False, "api_url": "prod.com"}
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

config = Config()


In [ ]:
class Logger:
    _instance = None

    def __init__(self):
        self.level:str = ""
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.level = "INFO"
        return cls._instance
    
    def log(self, msg):
        print(f"[{self.level}] {msg}")

class TempLogLevel:
    def __init__(self, level):
        self.temp_level = level
        self.logger = Logger()
    
    def __enter__(self):
        self.old_level = self.logger.level
        self.logger.level = self.temp_level
        return self.logger
    
    def __exit__(self, *args):
        self.logger.level = self.old_level

# Usage:
logger = Logger()
logger.log("Normal message")  # [INFO] Normal message

with TempLogLevel("DEBUG"):
    logger.log("Debug info")   # [DEBUG] Debug info

logger.log("Back to normal") # [INFO] Back to normal


[INFO] Normal message
[DEBUG] Debug info
[INFO] Back to normal
